In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from gurobipy import *


In [2]:
#Q = 200  # vehicle capacity
no_of_vehicles = 1# free to modify
no_of_customers =15

In [3]:
df = pd.read_csv(r'MVRP_400N.txt', sep = " ")


In [4]:
df = df.iloc[0:no_of_customers+1]

In [5]:
df

,Node,X,Y,Demand,Profit
0,1,0.0000,0.00,NaN,NaN
1,2,5.7735,0.00,40.0,16.0
2,3,2.8867,5.00,40.0,16.0
3,4,-2.8868,5.00,40.0,16.0
4,5,-5.7735,0.00,40.0,16.0
5,6,-2.8867,-5.00,40.0,16.0
6,7,2.8868,-5.00,40.0,16.0
7,8,8.6603,5.00,40.0,24.0
8,9,0.0000,10.00,40.0,24.0
9,10,-8.6603,5.00,40.0,24.0


In [6]:
Y, X = list(df["Y"]), list(df["X"])


In [7]:
coordinates = np.column_stack((X, Y))
coordinates

array([[  0.    ,   0.    ],
       [  5.7735,   0.    ],
       [  2.8867,   5.    ],
       [ -2.8868,   5.    ],
       [ -5.7735,   0.    ],
       [ -2.8867,  -5.    ],
       [  2.8868,  -5.    ],
       [  8.6603,   5.    ],
       [  0.    ,  10.    ],
       [ -8.6603,   5.    ],
       [ -8.6603,  -5.    ],
       [  0.    , -10.    ],
       [  8.6603,  -5.    ],
       [  5.3405,   0.75  ],
       [  3.3198,   4.25  ],
       [  6.4952,  -1.25  ]])

In [8]:
Demand = list(df["Demand"])

In [9]:
n = len(coordinates)

In [10]:
depot, customers = coordinates[0, :], coordinates[1:, :]

In [11]:
M = 100**100  # big number

In [12]:
m = Model("MVRP")

Academic license - for non-commercial use only - expires 2022-03-14
Using license file C:\Users\dilip\gurobi.lic


In [13]:
x, y, z = {}, {}, {}

In [14]:
n

16

In [15]:
L=n-1

In [16]:
dist_matrix = np.empty([n, n])
for i in range(len(X)):
    for j in range(len(Y)):
        '''variable_1: X[i,j] =(0,1), i,j = Nodes'''
        x[i, j] = m.addVar(vtype=GRB.BINARY, name="x%d,%d" % (i, j))
        dist_matrix[i, j] = np.sqrt((X[i] - X[j]) ** 2 + (Y[i] - Y[j]) ** 2)
        if i == j:
            dist_matrix[i, j] = M  ## big 'M'
        continue
m.update()

In [17]:
u_vars = {}
for i in range(n):
    u_vars[i] = m.addVar(vtype=gurobipy.GRB.INTEGER, name='u_' + str(i))
m.update()

In [18]:
for i in range( n):
        for j in range( n):
            m.addConstr(u_vars[i] - u_vars[j] + L * x[i, j]  <= L - 1)
m.update()

In [19]:
# '''variable_2: y[j] = cumulative demand covered'''
# for j in range(n):
#     y[j] = m.addVar(vtype=GRB.INTEGER, name="y%d" % (j))   # cumulative demand satisfied variable
#     z[j] = m.addVar(vtype=GRB.INTEGER, name="z%d" % (j))   # cumulative time variable
# m.update()


In [20]:
''''constraint_1: sum(x[i,j]) = 1, for i = 1,2,...,32'''  # vehicles leaving each customer node
for i in range(n - 1):
    m.addConstr(quicksum(x[(i + 1, j)] for j in range(n)) == 1)
m.update()

In [21]:
''' constraint_2: sum(x[i,j] =1 for j = 1,2,.....,32)'''  # vehicles arriving to each customer node
for j in range(n - 1):
    m.addConstr(quicksum(x[(i, j + 1)] for i in range(n)) == 1)
m.update()

In [22]:
'''constraint_3: sum(x[0,j]) = 5'''  # vehicles leaving depot
m.addConstr(quicksum(x[(0, j)] for j in range(n)) == no_of_vehicles)
m.update()

In [23]:
'''constraint_4: sum(x[i,0]) = 5'''  # vehicles arriving to depot
m.addConstr(quicksum(x[(i, 0)] for i in range(n)) == no_of_vehicles)
m.update()

In [24]:
# for i in range(n):  # Constraint 5, formula 7
#     for j in range(n):
#         m.addConstr((x[i, j] + x[j, i]) <= 1)
# m.update

In [25]:
# ''' Either of the constraint_5 or the constrain_6 can eliminate sub-tours independently'''

# '''constraint_5: capacity of vehicle and also eliminating sub-tours'''
# for j in range(n - 1):
#     m.addConstr(y[j + 1] <= Q)
#     m.addConstr(y[j + 1] >= Demand[j + 1])
#     for i in range(n - 1):
#         m.addConstr(y[j + 1] >= y[i + 1] + Demand[j + 1] * (x[i + 1, j + 1]) - Q * (1 - (x[i + 1, j + 1])))
# m.update()

In [ ]:
'''objective function'''
m.setObjective(quicksum(quicksum(x[(i, j)]*dist_matrix[(i, j)] for j in range(n)) for i in range(n)),GRB.MINIMIZE)
m.update()

'''optimize'''
m.optimize()


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 288 rows, 272 columns and 1248 nonzeros
Model fingerprint: 0x2db4e915
Variable types: 0 continuous, 272 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [9e-01, 1e+100]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 16 rows and 16 columns
Presolve time: 0.01s
Presolved: 272 rows, 256 columns, 1200 nonzeros
Variable types: 0 continuous, 256 integer (240 binary)

Root relaxation: objective 7.857132e+01, 74 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   78.57132    0   31          -   78.57132      -     

In [ ]:
sol_y, sol_x, sol_z = m.getAttr('x', y), m.getAttr('x', x), m.getAttr('x', z)

In [ ]:
X, Y, Z = np.empty([n, n]), np.empty([n]), np.empty([n])


In [ ]:
for i in range(n):
#     Y[i] = sol_y[i]
#     Z[i] = sol_z[i]
    for j in range(n):
        X[i, j] = int(sol_x[i, j])

In [ ]:
print('\nObjective is:', m.objVal)
print('\nDecision variable X (binary decision of travelling from one node to another):\n', X.astype('int32'))

In [ ]:
from_node = []
to_node = np.empty([n, n])

In [ ]:
for v in m.getVars():
    from_node.append(v.x)

In [ ]:
for i in range(n):
    for j in range(n):
        to_node[i, j] = from_node[n * i + j]

In [ ]:
print('\nDistance Matrix (dij):\n', pd.DataFrame(dist_matrix).astype('int64'))
print('\nDecision (Xij):\n', pd.DataFrame(to_node).astype('int64'))

In [ ]:
# to node:Decision matrix
I = []  #Row of decision matrix
J = []  # column of decision matrix
for i in range(n):
    for j in range(n):
        if to_node[i, j] > 0.5:
            I.append(i)
            J.append(j)

In [ ]:
print(np.array(I))
print(np.array(J))

In [ ]:
all_routes = {}
cnt = 0

In [ ]:
for i in range(no_of_vehicles):
    end = J[i]  # Take out the J[0] of (0,J[0])
    route = [0, end]  # Initial route=[0,J[0]]
    while end != 0:
        start_index = I.index(end)  # (# Find the index corresponding to J[0] in I[], which is the start_index of the next value)
        end = J[start_index]  # (# Put this index, to J[] to find the corresponding value,)
        route.append(end)  #  (Add the value to the sub-route list route)
    all_routes[str(i)] = route  # (Multiple sub-paths constitute the total path expression)
    # Output all_routes, including each sub-route
print('all sub route trace:')
print(all_routes)

In [ ]:
sub_route_total_distance = {}

for i in range(no_of_vehicles):
    prev_index = 0
    # print('\n')
    sub_route_total_distance[i] = 0
    for j in range(len(all_routes[str(i)])):
        cur_index = all_routes[str(i)][j]
        # print('dist[%d,%d]=%d' % (prev_index, cur_index, dist_matrix[prev_index][cur_index]))
        sub_route_total_distance[i] += dist_matrix[prev_index][cur_index]
        prev_index = cur_index
print('all sub route total distance sum:')
print(sub_route_total_distance)


In [ ]:
coordinates

In [ ]:
X=[]
Y=[]
for i,value in enumerate (coordinates):
    X.append(coordinates[i][0])
    Y.append(coordinates[i][1])
    

In [ ]:
# Figure 1 Original WRSN sensor node and charger Wireless radar sensor network
plt.figure()
plt.scatter(X[1:], Y[1:], marker='o', color='blue')
plt.scatter(X[0], Y[0], marker='^', color='blue')
plt.xlabel('x-coordinate(m)')
plt.ylabel('y-coordinate(m)')
# for i in range(no_of_vehicles):
#     plt.scatter(X[0] - 20 + i, Y[0] - 10, marker='^', color='red')
plt.title('Original sensors and chargers positions')


In [ ]:
# Figure 2 Trajectory after MTSP solution
plt.figure()
plt.scatter(X[1:], Y[1:], marker='o', color='blue')
plt.scatter(X[0], Y[0], marker='^', color='blue')
plt.xlabel('x-coordinate(m)')
plt.ylabel('y-coordinate(m)')
plt.title('Total min distance %g (m)' % m.objVal)
# plt.plot([XX1, XX2], [YY1, YY2])
for index in all_routes:
    route_list = all_routes[index]
    x = []
    y = []
    for i in route_list:
        x.append(X[i])
        y.append(Y[i])
    plt.plot(x, y)
plt.show()

In [ ]:
def plot_tours(solution_x):
    tours = [[i, j] for i in range(solution_x.shape[0]) for j in range(solution_x.shape[1]) if solution_x[i, j] ==1]
    for t, tour in enumerate(tours):
        plt.plot(df["X"][tour], df["Y"][tour], color = "black", linewidth=0.5)
    plt.scatter(df["X"][1:], df["Y"][1:], marker = 'x', color = 'g', label = "customers")
    plt.scatter(df["X"][0], df["Y"][0], marker = "o", color = 'b', label = "depot")
    plt.xlabel("X"), plt.ylabel("Y"), plt.title("Tours"), plt.legend(loc = 4)
    plt.show()

plot_tours(X)

In [ ]:
df["X"]

In [ ]:
tours = [[i, j] for i in range(X.shape[0]) for j in range(X.shape[1]) if X[i, j] ==1]